[사용]
pykospacing(맞춤법), kkma.sentence(or kss)

[시도]
kss(문장분리) / mecab, kkma, okt pos

[순서]
자막 다운 -> 띄어쓰기 제거, kkma.sentence -> 야매 문장 분리 -> 띄어쓰기 제거, spacing
-kss 시도를 해봤지만, 문장 부호가 없으면 시간이 많이 걸리고 모든 문장으로 안 나뉘어지는 단점 // 다시 해보니 0.3초 뭐지...
-spacing을 먼저 해봤으나 맞춤법 전문이라 제대로 안 나뉘어짐 ('~요' 인식이 잘 안 됨)
-꼬꼬마로 문장을 어느정도 나누고, 문장 부호를 추가하니 그나마 나아짐




In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

srt = YouTubeTranscriptApi.get_transcript("3WWC9cgIJbc", languages=['ko']) #한글로, 딕셔너리 구조

# 텍스트 파일로 저장할 경우
# with open("subtitles.txt", "w", encoding='utf-8') as f:  
#     for i in srt:
#         # writing each element of srt on a new line
#         f.write("{}\n".format(i))

text = ''

for i in range(len(srt)):
    text += srt[i]['text'] + ' '
    
text_ = text.replace(' ','')

In [18]:
from konlpy.tag import Kkma

#문장 분리 / kss 사용해도 무방
kkma = Kkma()

text_sentences = kkma.sentences(text_)
text_sentences[:6]

['4[ 음악] 최근 한국 음식의 세계적 위상이 날로 상승하고 있습니다',
 '특히 이런 케이프들 대표주자 격인 우리나라의 불고기는 이제 우리가 상상할 수 있는 범위를 넘어선 곳까지 진출하고 있는데요 세계인들에게 낯선 음식 중 하나에 불과했던 한식이 싸늘한 시선을 받던 몇 년 증권당이 이제 한국 음식은 책의 어떤 가를 음식과 견주어도 손색이 없는 전세계에 별미로 자리잡고 있습니다',
 '실제로 2014년에는 한식의 세계화를 위해 세계 각국의 주요 언론사에 불고기 홍보를 했던 우리나라가 국내외를 막론하고 쏟아지는 비판을 피하지 못했던 사건이 있었는데요 당시 한국의 먹거리를 세계에 알리자는 취지에서 홍보를 계획한 우리나라는 세계 적인 스포츠스타로 발돋움하고 있던 추신 추선수를 내세워 불고기라는 멘트를 넣은 광고를 전세기언론의 찍기 시작했고 이에 대한 반응은 냉담하기 그지없었습니다',
 '다소 때 용을 해석하기 어려운 의문투성이 광고의미 국의 대표적인 공영 방송인 npr의 루이스 클라 맨 슨 은한 국 이하고 있는 광고 민 도저히 뜻을 이해할 수가 없고 독자들에게 혼란을 주고 있다는 비판을 하기도 했죠',
 '이에 더 의이 광고를 접한 미국인들은 불고기가 대체 어떤 나라말이냐',
 '아프리카 원주민의 언어인 것 같다며 우리나라의 의도와는 전혀 다른 반응을 보였는데요 이러한 반응이 우리나라에 전해 진짜 당시 네티즌들은 왜 굳이 나서 서 나라 망신을 시키는 것이냐']

In [11]:
import pandas as pd

#종결 단어
lst = ['죠','다','요','시오', '습니까','십니까','됩니까','옵니까','뭡니까',]

df = pd.read_csv('not_verb.csv',encoding='utf-8')
not_verb = df.stop.to_list()

#단어 단위로 끊기
text_all = ' '.join(text_sentences).split(' ')

for n in range(len(text_all)) :
    i = text_all[n]
    if len(i) == 1 : #한글자일 경우 추가로 작업x
        continue
    
    else :
        for j in lst : #종결 단어
            #질문형
            if j in lst[4:]:
                i += '?'
            
            #명령형                
            elif j == '시오' :
                i += '!'
            
            #마침표    
            else :
                if i in not_verb : #특정 단어 제외
                    continue
                else :        
                    if j == i[len(i)-1] : #종결
                            text_all[n] += '.'
                            print(text_all[n], end = '/ ')
                
# print()            
# print(text_all)


있습니다./ 있는데요./ 있습니다./ 있었는데요./ 그지없었습니다./ 했죠./ 보였는데요./ 냈습니다./ 것인데요./ 되었습니다./ 곳인데요./ 것이죠./ 나갔는데요./ 보였습니다./ 시작했는데요./ 뜨거웠습니다./ 맛이다./ 맛이었다./ 했죠./ 지였는데요./ 달렸습니다./ 졌다./ 버렸어요./ 보였죠./ 것입니다./ 있는데요./ 됐습니다./ 때문인데요./ 것입니다./ 있었죠./ 있었습니다./ 있는데요./ 썼죠./ 있었습니다./ 있죠./ 일이었는데요./ 보였습니다./ 했다./ 고하는데요./ 되었죠./ 다고합니다./ 것이죠./ 있는데요./ 쉽습니다./ 마치겠습니다./ 

In [13]:
from pykospacing import Spacing

spacing = Spacing()
text_all_in_one = ' '.join(text_all)
# print(spacing(text_all_in_one))
# print()
print(spacing(text_all_in_one.replace(' ',''))) #성능이 더 좋음 / . 다음에 띄어쓰기 없어진게 몇 개 있음

4 [음악] 최근 한국 음식의 세계적 위상이 날로 상승하고 있습니다. 특히 이런 케이프들 대표주자 격인 우리나라의 불고기는 이제 우리가 상상할 수 있는 범위를 넘어선 곳까지 진출하고 있는데요.세계인들에게 낯선 음식 중 하나에 불과했던 한식이 싸늘한 시선을 받던 몇 년 증권당이 이제 한국 음식은 책의 어떤가를 음식과 견주어도 손색이 없는 전 세계에 별미로 자리잡고 있습니다. 실제로 2014년에는 한식의 세계화를 위해 세계 각국의 주요 언론사에 불고기 홍보를 했던 우리나 라가 국내외를 막론하고 쏟아지는 비판을 피하지 못했던 사건이 있었는데요. 당시 한국의 먹거리를 세계에 알리자는 취지에서 홍보를 계획한 우리나라는 세계적인 스포츠 스타로 발돋움하고 있던 추신 추선수를 내세워 불고기라는 멘트를 넣은 광고를 전세기 언론의 찍기 시작했고 이에 대한 반응은 냉담하기 그지 없었습니다. 다소 때 용을 해석하기 어려운 의문투성이 광고의 미국의 대표적인 공영방송인 npr의 루이스 클라맨슨은 한국이 하고 있는 광고민도저히 뜻을 이해할 수가 없 고독자들에게 혼란을 주고 있다는 비판을 하기도 했죠.이에 더의 이 광고를 접한 미국인들은 불고기가 대체 어떤 나라 말이냐 아프리카 원주민의 언어인 것 같다며 우리나라의 의도와는 전혀 다른 반응을 보였는데요.이러한 반응이 우리나라에 전해진짜 당시 네티즌들은 왜 굳이 나서서 나라 망신을 시키는 것이냐 외국인들 보기가 부끄러울 따름이다라며 해외인들과 크게 다르지 않은 날이 선 댓글을 달고 냈습니다. 한국을 세계에 알리자는 취지의 자체는 주었으나 그 설득력이 뭐 쪽이나 부족했던 광고했던 것인데요. 그러나 불과 7년이 지난 지금 이 사건의 주인공이었던 불고기는 세계인들에게 전혀 다른 인식의 음식으로 탈바꿈하게 되었습니다.빌보드 차트 2위를 기록하며 공전의 히트를 친 4일을 기점으로 뻗어 나간 한류의 문화는 우리나라 음식들의 인지도 역시 새롭게 갱신시키는 데 성공한 곳인데요.특유의 달달하고 도 짭조름한 육질이 연하지만 식감도 뛰어난 불고기는 처음 먹어보

In [52]:
#문자열 재정비
text_split= text_all_in_one.replace('[ 음악]','').split('.')
text2one = []
for t in text_split:
    text2one.append(t.lstrip())

# print(text2one)    
tt ='. '.join(text2one)
print(tt)


4 최근 한국 음식의 세계적 위상이 날로 상승하고 있습니다. 특히 이런 케이프들 대표주자 격인 우리나라의 불고기는 이제 우리가 상상할 수 있는 범위를 넘어선 곳까지 진출하고 있는데요. 세계인들에게 낯선 음식 중 하나에 불과했던 한식이 싸늘한 시선을 받던 몇 년 증권당이 이제 한국 음식은 책의 어떤 가를 음식과 견주어도 손색이 없는 전세계에 별미로 자리잡고 있습니다. 실제로 2014년에는 한식의 세계화를 위해 세계 각국의 주요 언론사에 불고기 홍보를 했던 우리나라가 국내외를 막론하고 쏟아지는 비판을 피하지 못했던 사건이 있었는데요. 당시 한국의 먹거리를 세계에 알리자는 취지에서 홍보를 계획한 우리나라는 세계 적인 스포츠스타로 발돋움하고 있던 추신 추선수를 내세워 불고기라는 멘트를 넣은 광고를 전세기언론의 찍기 시작했고 이에 대한 반응은 냉담하기 그지없었습니다. 다소 때 용을 해석하기 어려운 의문투성이 광고의미 국의 대표적인 공영 방송인 npr의 루이스 클라 맨 슨 은한 국 이하고 있는 광고 민 도저히 뜻을 이해할 수가 없고 독자들에게 혼란을 주고 있다는 비판을 하기도 했죠. 이에 더 의이 광고를 접한 미국인들은 불고기가 대체 어떤 나라말이냐 아프리카 원주민의 언어인 것 같다며 우리나라의 의도와는 전혀 다른 반응을 보였는데요. 이러한 반응이 우리나라에 전해 진짜 당시 네티즌들은 왜 굳이 나서 서 나라 망신을 시키는 것이냐 외국인들 보기가 부끄러울 따름이 다라며 해외 인들과 크게 다르지 않은 날이 선 댓 글을 달고 냈습니다. 한국을 세계에 알리자는 취지의 자체는 주었으나 그 설득력이 뭐 쪽이나 부족했던 광고했던 것인데요. 그러나 불과 7년이 지난 지금이사 건의 주인공이었던 불고기는 세계인들에게 전혀 다른 인식의 음식으로 탈바꿈하게 되었습니다. 빌 보드 차트 2 위를 기록하며 공전의 히트를 친 4일을 기점으로 뻗어 나간 한류의 문화는 우리나라 음식들의 인지도 역시 새롭게 갱신시키는데 성공한 곳인데요. 특유의 달 달하고도 짭조름한 육질이 연하지 만식 감도 뛰어난 불고기

ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ[시도]ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ

In [ ]:
from pykospacing import Spacing

sent = '김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.'
new_sent = sent.replace(" ", '') # 띄어쓰기가 없는 문장 임의로 만들기

spacing = Spacing()
kospacing_sent = spacing(new_sent) 

print(new_sent)
print(kospacing_sent)

김철수는극중두인격의사나이이광수역을맡았다.철수는한국유일의태권도전승자를가리는결전의날을앞두고10년간함께훈련한사형인유연재(김광수분)를찾으러속세로내려온인물이다.
김철수는 극중 두 인격의 사나이 이광수 역을 맡았다. 철수는 한국 유일의 태권도 전승자를 가리는 결전의 날을 앞두고 10년간 함께 훈련한 사형인 유연재(김광수 분)를 찾으러 속세로 내려온 인물이다.


In [32]:
from youtube_transcript_api import YouTubeTranscriptApi
from pykospacing import Spacing

srt = YouTubeTranscriptApi.get_transcript("3WWC9cgIJbc", languages=['ko']) #한글로
# print(srt) # 리스트 구조 내 딕셔너리
# print(srt[2])
# {'text': '4', 'start': 0.0, 'duration': 10.18}

text = ''

for i in range(len(srt)):
    text += srt[i]['text'] + ' '
    
text_ = text.replace(' ','')

spacing = Spacing()
spacing_text = spacing(text_)  #4초 정도?

print(spacing_text)

4 [음악] 최근 한국 음식의 세계적 위상이 날로 상승하고 있습니다 특히 이런 케이프들 대표주자 격인 우리나라의 불고기는 이제 우리가 상상할 수 있는 범위를 넘어선 곳까지 진출하고 있는데 요세계인들에게 낯선 음식 중 하나에 불과했던 한식이 싸늘한 시선을 받던 몇 년 증권당이 이제 한국 음식은 책의 어떤가를 음식과 견주어도 손색이 없는 전 세계에 별미로 자리잡고 있습니다 실제로 2014년에는 한식의 세계화를 위해 세계 각국의 주요 언론사에 불고기 홍보를 했던 우리나라가 국 내외를 막론하고 쏟아지는 비판을 피하지 못했던 사건이 있었는데 요 당시 한국의 먹거리를 세계에 알리자는 취지에서 홍보를 계획한 우리나라는 세계적인 스포츠 스타로 발돋움하고 있던 추신 추선수를 내세워 불고기라는 멘트를 넣은 광고를 전세기 언론의 찍기 시작했고 이에 대한 반응은 냉담하기 그지 없었습니다 다소 때 용을 해석하기 어려운 의문투성이 광고의 미국의 대표적인 공영방송인 npr의 루이스 클라맨슨은 한국이 하고 있는 광고민도저히 뜻을 이해할 수가 없고 독자들에 게 혼란을 주고 있다는 비판을 하기도 했죠 이에 더의 이 광고를 접한 미국인들은 불고기가 대체 어떤 나라 말이냐 아프리카 원주민의 언어인 것 같다며 우리나라의 의도와는 전혀 다른 반응을 보였는데 요 이러한 반응이 우리나라에 전해진짜 당시 네티즌들은 왜 굳이 나서서 나라 망신을 시키는 것이냐 외국인들 보기가 부끄러울 따름이다라며 해외인들과 크게 다르지 않은 날이 선 댓글을 달고 냈습니다 한국을 세계에 알리자는 취지의 자체는 주었으나 그 설득력이 뭐 쪽이나 부족했던 광고했던 것인데요 그러나 불과 7년이 지난 지금 이 사건의 주인공이었던 불고기는 세계인들에게 전혀 다른 인식의 음식으로 탈바꿈하게 되었습니다 빌보드 차트 2위를 기록하며 공전의 히트를 친 4일을 기점으로 뻗어 나간 한류의 문화는 우리나라 음식들의 인지도 역시 새롭게 갱신시키는 데 성공한 곳인데요 특유의 달달하고 도 짭조름한 육질이 연하지만 식감도 뛰어난 불고기는 처음 먹어보는 이

In [30]:
from youtube_transcript_api import YouTubeTranscriptApi
from pykospacing import Spacing

srt = YouTubeTranscriptApi.get_transcript("k_zc0tWljk4", languages=['ko']) #한글로
# print(srt) # 리스트 구조 내 딕셔너리
# print(srt[2])
# {'text': '4', 'start': 0.0, 'duration': 10.18}

text = ''

for i in range(len(srt)):
    text += srt[i]['text'] + ' '



In [33]:
import kss

for t in kss.split_sentences(text):
    print(t)
    
#213초나 걸릴정도로 비효율적???? / 다시 해보니 0.3초

4 [음악] 최근 한국 음식 의 세계적 위상이 날로 상승하고 있습니다
특히 이런 케이프 들 대표주자 격인 우리나라의 불고기는 이제 우리가 상상할 수 있는 범위를 넘어선 곳까지 진출하고 있는데요
세계인들에게 낯선 음식 중 하나에 불과했던 한식이 싸늘한 시선을 받던 몇 년 증권 당이 이제 한국 음식은 책의 어떤가를 음식과 견주어도 손색이 없는 전세계에 별미로 자리잡고 있습니다
실제로 2014년에는 한식의 세계화를 위해 세계 각국의 주요 언론사에 불고기 홍보를 했던 우리나라가 국내외를 막론하고 쏟아지는 비판을 피하지 못했던 사건이 있었는데요
당시 한국의 먹거리를 세계에 알리자는 취지에서 홍보를 계획한 우리나라는 세계적인 스포츠 스타로 발돋움 하고 있던 추신 추 선수를 내세워 불고기 라는 멘트를 넣은 광고를 전세기 언론의 찍기 시작했고 이에 대한 반응은 냉담 하기 그지 없었습니다
다소 때 용을 해석하기 어려운 의문 투성이 광고의 미국의 대표적인 공영방송인 npr 의 루이스 클라 맨슨은 한국이 하고 있는 광고 민 도저히 뜻을 이해할 수가 없고 독자들에게 혼란을 주고 있다는 비판을 하기도 했죠
이에 더 의 이 광고를 접한 미국인들은 불고기가 대체 어떤 나라 말이냐 아프리카 원주민의 언어인 것 같다며 우리나라의 의도와는 전혀 다른 반응을 보였는데요
이러한 반응이 우리나라에 전해진 짜 당시 네티즌들은 왜 굳이 나서서 나라 망신을 시키는 것이냐 외국인들 보기가 부끄러울 따름이다 라며 해외 인들과 크게 다르지 않은 날이 선 댓글을 달고 냈습니다 한국을 세계에 알리자 는 취지의 자체는 주었으나 그 설득력이 뭐 쪽이나 부족했던 광고했던 것인데요
그러나 불과 7년이 지난 지금 이 사건 의 주인공이었던 불고기는 세계인들에게 전혀 다른 인식의 음식 으로 탈바꿈하게 되었습니다
빌보드 차트 2위를 기록하며 공전의 히트를 친 4일을 기점으로 뻗어나간 한류의 문화는 우리나라 음식들의 인지도 역시 새롭게 갱신 시키는데 성공한 곳인데요
특유의 달달하고 도 짭조름한 육질이 연하지만 식감도 뛰어난 불

In [2]:
from konlpy.tag import Kkma
kkma = Kkma()
tagger = kkma.pos(spacing_text)
# print(tagger)

In [16]:
text_sentences = kkma.sentences(spacing_text)
text_sentences

['4 [ 음악] 최근 한국 음식의 세계적 위상이 날로 상승하고 있습니다',
 '특히 이런 케이프들 대표주자 격인 우리나라의 불고기는 이제 우리가 상상할 수 있는 범위를 넘어선 곳까지 진출하고 있는데 요 세계인들에게 낯선 음식 중 하나에 불과했던 한식이 싸늘한 시선을 받던 몇 년 증권당이 이제 한국 음식은 책의 어떤 가를 음식과 견주어도 손색이 없는 전 세계에 별미로 자리잡고 있습니다',
 '실제로 2014년에는 한식의 세계화를 위해 세계 각국의 주요 언론사에 불고기 홍보를 했던 우리나라가 국 내외를 막론하고 쏟아지는 비판을 피하지 못했던 사건이 있었는데 요 당시 한국의 먹거리를 세계에 알리자는 취지에서 홍보를 계획한 우리나라는 세계적인 스포츠 스타로 발돋움하고 있던 추신 추선수를 내세워 불고기라는 멘트를 넣은 광고를 전세기 언론의 찍기 시작했고 이에 대한 반응은 냉담하기 그지 없었습니다',
 '다 소 때 용을 해석하기 어려운 의문투성이 광고의 미국의 대표적인 공영 방송인 npr의 루이스 클라 맨 슨 은 한국이 하고 있는 광고 민 도저히 뜻을 이해할 수가 없고 독자들에 게 혼란을 주고 있다는 비판을 하기도 했죠',
 '이에 더의 이 광고를 접한 미국인들은 불고기가 대체 어떤 나라 말이냐',
 '아프리카 원주민의 언어인 것 같다며 우리나라의 의도와는 전혀 다른 반응을 보였는데 요 이러한 반응이 우리나라에 전해 진짜 당시 네티즌들은 왜 굳이 나서 서 나라 망신을 시키는 것이냐',
 '외국인들 보기가 부끄러울 따름이 다라며 해외 인들과 크게 다르지 않은 날이 선 댓 글을 달고 냈습니다',
 '한국을 세계에 알리자는 취지의 자체는 주었으나 그 설득력이 뭐 쪽이나 부족했던 광고했던 것인데요 그러나 불과 7년이 지난 지금 이 사건의 주인공이었던 불고기는 세계인들에게 전혀 다른 인식의 음식으로 탈바꿈하게 되었습니다',
 '빌 보드 차트 2위를 기록하며 공전의 히트를 친 4일을 기점으로 뻗어 나간 한류의 문화는 우리나라 음식들의 인지도 역시 새롭게 갱신시키는 데 성공한 곳인

In [53]:
from konlpy.tag import Kkma, Okt
import pandas as pd

df = pd.read_csv('not_verb.csv',encoding='utf-8')
df.info()
not_verb = df.stop.to_list()
kkma = Kkma()

# 꼬꼬마로 해봤으나 형태소 분류를 세분화해서 명사인지 동사인지 판단 힘듦
# for i in not_verb :
#     print(kkma.pos(i))
    
# okt는 pos 간단하게 verb noun을 표시
okt = Okt()
s ='이러한 불고기는 초든 보다. 간편하게 접할 수 있는 레 트로트형태로'
s2 = '실제로 2014년에는 한식의 세계화를 위해 세계 각국의 주요. 언론사에 불고기 홍보를 했던 우리나라가 국내외를 막론하고 쏟아지는 비판을 피하지 못했던 사건이 있었는데요. '
print(okt.pos(s2))

print()
for i in not_verb :
    print(okt.pos(i)[0][1], end=' / ')

# 것인데요. 시작했습니다. 이런 단어에는 문제가 있음

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47 entries, 0 to 46
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   stop    47 non-null     object
dtypes: object(1)
memory usage: 504.0+ bytes
[('실제', 'Noun'), ('로', 'Josa'), ('2014년', 'Number'), ('에는', 'Foreign'), ('한식', 'Noun'), ('의', 'Josa'), ('세계', 'Noun'), ('화', 'Suffix'), ('를', 'Josa'), ('위해', 'Noun'), ('세계', 'Noun'), ('각국', 'Noun'), ('의', 'Josa'), ('주요', 'Noun'), ('.', 'Punctuation'), ('언론사', 'Noun'), ('에', 'Josa'), ('불고기', 'Noun'), ('홍보', 'Noun'), ('를', 'Josa'), ('했던', 'Verb'), ('우리나라', 'Noun'), ('가', 'Josa'), ('국내외', 'Noun'), ('를', 'Josa'), ('막론', 'Noun'), ('하고', 'Josa'), ('쏟아지는', 'Verb'), ('비판', 'Noun'), ('을', 'Josa'), ('피하', 'Noun'), ('지', 'Josa'), ('못', 'VerbPrefix'), ('했던', 'Verb'), ('사건', 'Noun'), ('이', 'Josa'), ('있었는데요', 'Adjective'), ('.', 'Punctuation')]

Noun / Noun / Noun / Noun / Noun / Noun / Noun / Noun / Noun / Noun / Noun / Adjective / Noun / Noun / No